<h3>Scrape Case Details</h3>
From each case's page on CaseNet, collect the following information:
<ul>
    - Judge name <br>
    - Disposed status <br>
    - Charge Description <br>
    - Ordinance Number <br>
    - Law Enforcement Agency <br>
    - Civil judgments <br>
    - Garnishments and executions <br>
    </ul>

In [1]:
#import the necessary python packages
import sys

selenium_path = r'C:\Users\Kimbe\anaconda3\Lib\site-packages\selenium'
PYSOLR_PATH = r'c:\users\kimbe\appdata\local\programs\python\python312\lib\site-packages'

if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)

if not selenium_path in sys.path:
    sys.path.append(selenium_path)

import pandas
import numpy
import requests
import bs4
import re
import undetected_chromedriver as uc
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

import re

In [2]:
suffix_header = '#header'
suffix_parties = '#party'
suffix_docket = '#docket'
suffix_charges = '#charges'
suffix_service = '#service'
suffix_hearings = '#scheduled'
suffix_judgment = '#judgment'
suffix_garnishments = '#garnishment'


In [3]:
#read the links
cases = pandas.read_csv(r"C:\Users\Kimbe\Documents\Data Science\Sample Data - Week of Jan 2, 2025.csv")

In [4]:
sample_link = cases.iloc[0,5] #first row, fifth column
sample_case_num = cases.iloc[0,1]
print(sample_case_num)

sample_links = [(sample_link + suffix_header), (sample_link + suffix_parties), (sample_link + suffix_docket), (sample_link + suffix_charges), (sample_link + suffix_service), (sample_link + suffix_hearings), (sample_link + suffix_judgment), (sample_link + suffix_garnishments)]
sample_links = pandas.DataFrame({'Links': sample_links})
sample_links['Case_Num'] = sample_case_num
print(sample_links)

240286808
                                               Links   Case_Num
0  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
1  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
2  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
3  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
4  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
5  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
6  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808
7  https://www.courts.mo.gov/cnet/cases/newHeader...  240286808


In [5]:
#Launch the headless browser
driver = uc.Chrome()
#sample_link = 'https://www.courts.mo.gov/cnet/cases/newHeader.do?inputVO.caseNumber=24SL-CR00046-01&inputVO.courtId=CT21#header'
#Navigate to the CaseNet URL
driver.get(sample_links.iloc[0,0])

#Give the page ample time to load
time.sleep(10)
wait = WebDriverWait(driver, 10)

html_data_judge = driver.page_source
soup_judge = bs4.BeautifulSoup(html_data_judge, 'html.parser')


In [6]:
objects = str(soup_judge.find_all(attrs={"class": 'col-lg-5 col-md-6 col-sm-6 col-xs-12 odd'})).replace('<strong>', '').replace('<div class="col-lg-5 col-md-6 col-sm-6 col-xs-12 odd', '').replace('">', '').replace('</strong>', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
values = str(soup_judge.find_all(attrs={"class": 'col-lg-7 col-md-6 col-sm-6 col-xs-12 even'})).replace('<div class="col-lg-7 col-md-6 col-sm-6 col-xs-12 even', '').replace('">', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
 
column_names = [x.strip() for x in objects.split(',')]

case_data = [x.strip() for x in values.split(',')]

#Clean up surname issues
if 'Judge/Commissioner Assigned' in column_names:
    case_data[0] = case_data[0] + ', ' + case_data[1]
    case_data.pop(1)

if 'Judge/Commissioner At Disposition' in column_names:
    first_name_index = len(case_data) - 1
    last_name_index = first_name_index - 1
    case_data[last_name_index] = case_data[last_name_index] + ', ' + case_data[first_name_index]
    case_data.pop(first_name_index)
    
if 'Financial Information' in column_names:
    fin_info_index = len(case_data)
    column_names.pop(fin_info_index)
    
case_details = pandas.DataFrame([case_data], columns = [column_names])

In [7]:
case_details

,Judge/Commissioner Assigned,Date Filed,Location,Case Type,Disposition
0,"MORRIS, RAPHAEL ODELL",01/02/2025,Pagedale Municipal,Municipal/ Ordinance - see Charges,Not Disposed


In [28]:
#How can I handle situations where a case has extra headers? example: Judge/Commissioner at Disposition
case_details['case_number'] = sample_case_num


SyntaxError: invalid syntax (Temp/ipykernel_17104/831296096.py, line 2)